In [1]:
# Load libraries for file handling and image crunching
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp

#import scipy.ndimage as ndicpu
import cupyx.scipy.ndimage as ndi

import seaborn as sns
import pandas as pd
# Set matplotlib backend
%matplotlib inline 

import cucim.skimage as skimage
# Import the os module
import os

#fancy gui viewer
import napari


In [2]:
# get imagej API
wdpath = os.getcwd()
import imagej
#ij = imagej.init()
ij = imagej.init('/home/linus/Postdoc/Fiji.app/')

# import some previously written conveniece functions
from MGlia_detect_utils import *

# grab a testimage
testimage = "C2-220421 otof_iba slide002 mouse195 005.tif"

testimagebrain = "320763 CNIC.tif"
dirpath = "./"

filepath = os.path.join(wdpath, testimage)

# load the image
from skimage.io import imread
img = imread(filepath)

In [3]:
# read the image with imagej
img = ij.io().open(filepath)

# the scale is in the coordinate part of this object
img = ij.py.from_java(img)
#extract the number as scalar, is in microns
planestep = float(img['pln'][1].to_numpy())
xystep = float(img['row'][1].to_numpy())

pixvol =planestep*xystep**2

print("xy scale is ", xystep, "micron per pixel")
print("z scale is ", planestep, "micron per pixel")
print("1 voxel is", pixvol, "microns cubed")

xy scale is  0.27500004812500844 micron per pixel
z scale is  0.3 micron per pixel
1 voxel is 0.02268750794062709 microns cubed


In [7]:
#convert img to cp array on GPU
gpu_img = cp.asarray(img)

In [10]:
# run a gaussian filter on GPU
gpu_filtered = ndi.filters.gaussian_filter(gpu_img, 5)

In [11]:
# define a cube of x microns as a footprint
# use the scale and floor division to find the number of pixels in each dimension to use
x = 30
foot = np.ones((3, # go plane-by plane, get to many rather than too few seeds
               int(x//xystep),
               int(x//xystep)))


In [48]:
from skimage.feature import peak_local_max
locmax = peak_local_max(cpu_filtered, min_distance=0, footprint = foot)

In [51]:
x = 30
foot = cp.ones((3, # go plane-by plane, get to many rather than too few seeds
               int(x//xystep),
               int(x//xystep)))

In [12]:
gpu_locmax = ndi.maximum_filter(gpu_filtered, footprint = foot)

gpu_candmask = gpu_locmax  == gpu_filtered

#dilate the seed for plotting
gpu_candmask_plot = ndi.binary_dilation(gpu_candmask, structure = cp.ones((3,10,10)))



In [14]:
viewer = napari.view_image(img)
#new_layer = viewer.add_image(cp.asnumpy(gpu_locmax), opacity = 0.2, colormap = "red")
new_layer = viewer.add_image(cp.asnumpy(gpu_candmask_plot), opacity = 0.5, colormap = "red")

array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]])